In [1]:
# !sudo pip install --upgrade -q transformers

In [2]:
from refractml import *
from refractml.constants import MLModelFlavours
# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
# payload = {"payload":[{"question":"top 2 segments and bottom 3 sub-category basis quantity","context":{
#     "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
#                 {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
#                 {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
#                 {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
#     "DATE VARIABLE": [
#         {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
# }}]}

In [4]:
# import json
# print(json.dumps(payload,indent=2))

In [5]:
# context ={
#     "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
#                 {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
#                 {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
#                 {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
#     "DATE VARIABLE": [
#         {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
# }

In [3]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [4]:
import time

In [5]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        self.model_loaded = None
        self.tokenizer = None
        self.model_name ="/data/mistral/query-to-mql/exp-10/nov-20/merged-model"
        self.date_input = {
            "start_date": "01/01/2020",
            "end_date": "15/09/2023"}

        from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
        import time
        import torch
        from huggingface_hub.utils import disable_progress_bars
        disable_progress_bars()
        if self.model_loaded is None:
            print("Loading mistral fine tuned model")
            # self.model_loaded = pipeline("text-generation", model= self.model_name, torch_dtype=torch.bfloat16, use_fast=False, device_map="auto",model_kwargs={"offload_folder":"offload"})
            self.model_loaded = AutoModelForCausalLM.from_pretrained(self.model_name,torch_dtype=torch.bfloat16,device_map="cuda")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True, use_fast=False)
            
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[{"question": "","context":""},{"question": "","context":""},...]

        :return: (n_inputs, payload's)

        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (len(raw_payload), raw_payload) 
    
    def pre_processing_fn(self,payload):
        return payload

    
    def prediction_fn(self,
                      model: Any,
                      input_query
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        user_query = input_query["question"]
        context = input_query["context"]
        date_input = input_query["date_input"]
        prompt = f"""Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]"""
        print("prompting has started")
        start = time.time()
        prompt = self.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        response = self.model_loaded.generate(input_ids=prompt, max_length= 500, pad_token_id=self.tokenizer.eos_token_id)
        # response = self.model_loaded(prompt, max_new_tokens=1000)#.to("cuda")
        print(f"Total time taken = {time.time()-start}")
        response = self.tokenizer.decode(response[0])
        # response = response[0]["generated_text"]
        output = response.split('[MQL]\n')[1]
        preds =  output.split('\n[/MQL]')[0]
        print("prediction is \n: ",preds)
        return [preds]

In [6]:
score_ = ScoreTemplateExample()
req = requests.Request()

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpz009f_56
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpz009f_56/_remote_module_non_scriptable.py


Loading mistral fine tuned model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

In [9]:
date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [8]:
req.json = {"payload":[{"question":"top 2 segments and bottom 3 sub-category basis quantity",
                        "context":context,
                       "date_input":date_input}]}
score_.score(None, req, dry_run=True)

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

prompting has started


OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB (GPU 0; 14.58 GiB total capacity; 14.15 GiB already allocated; 41.56 MiB free; 14.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
model="/data/mistral/query-to-mql/exp-10/nov-20/merged-model"
#model=score_.model_loaded

In [8]:
register_model(model,
               ScoreTemplateExample,
               "Mistral_fine_tuned_for_query_to_iMQL_v2",
               "Mistral_fine_tuned_for_query_to_iMQL_v2_model and input both on cuda explictly and max len 500",
               MLModelFlavours.pytorch,
               init_script="""sudo chmod -R 777 /home/mosaic-ai/.cache\\npip install torch \\npip install --upgrade transformers\\npip install accelerate"""
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "POST /registry/api/v1/ml-model/register HTTP/1.1" 200 3902
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/1c277dad-0388-4dbb-828a-826d631fd891 HTTP/1.1" 200 3895
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/1c277dad-0388-4dbb-828a-826d631fd891 HTTP/1.1"